In [3]:
# Step 1: Import libraries
import pandas as pd

# Step 2: Load all datasets (local paths assumed)
cms_df = pd.read_excel("cms_data.xlsx")  # Cleaned CMS data
income_df = pd.read_csv("us_income_zipcode.csv", encoding='latin1')
zip_df = pd.read_csv("uszips.csv", encoding='latin1')

# Step 3: Standardize ZIP code formatting for merging
cms_df['ZIP'] = cms_df['ZIP'].astype(str).str.zfill(5)
income_df['ZIP'] = income_df['ZIP'].astype(str).str.zfill(5)
zip_df.rename(columns={'zip': 'ZIP'}, inplace=True)
zip_df['ZIP'] = zip_df['ZIP'].astype(str).str.zfill(5)

# Step 4: Select relevant columns for merging
income_df = income_df[['ZIP', 'Households Median Income (Dollars)', 'Households Mean Income (Dollars)']]
zip_df = zip_df[['ZIP', 'lat', 'lng', 'city', 'state_name', 'population']]

# Step 5: Merge all datasets together on ZIP code
merged_df = cms_df.merge(income_df, on='ZIP', how='left')
merged_df = merged_df.merge(zip_df, on='ZIP', how='left')

# Step 6: Save the cleaned and merged dataset
merged_df.to_csv("cms_with_income_and_location.csv", index=False)

# Step 7: Preview merged dataset
print("Merged dataset saved as 'cms_with_income_and_location.csv'")
print(merged_df.head())


Merged dataset saved as 'cms_with_income_and_location.csv'
                MEDICAL CENTER NAME                 ADDRESS    CITY STATE  \
0  Southeast Alabama Medical Center  1108 Ross Clark Circle  Dothan    AL   
1  Southeast Alabama Medical Center  1108 Ross Clark Circle  Dothan    AL   
2  Southeast Alabama Medical Center  1108 Ross Clark Circle  Dothan    AL   
3  Southeast Alabama Medical Center  1108 Ross Clark Circle  Dothan    AL   
4  Southeast Alabama Medical Center  1108 Ross Clark Circle  Dothan    AL   

     ZIP                                        PROCEDURE  \
0  36301  Level 2 Excision/ Biopsy/ Incision and Drainage   
1  36301  Level 2 Excision/ Biopsy/ Incision and Drainage   
2  36301  Level 2 Excision/ Biopsy/ Incision and Drainage   
3  36301  Level 2 Excision/ Biopsy/ Incision and Drainage   
4  36301  Level 2 Excision/ Biopsy/ Incision and Drainage   

   Avg_Tot_Sbmtd_Chrgs  Avg_Mdcr_Alowd_Amt  Avg_Mdcr_Pymt_Amt      Source  \
0          9575.005714         103

In [5]:
# Step 6: Create Affordability Score (lower is better)
merged_df['Affordability_Score'] = merged_df['Avg_Mdcr_Pymt_Amt'] / merged_df['Households Median Income (Dollars)']

# Step 7: Save the cleaned and merged dataset with score
merged_df.to_csv("cms_with_income_and_location.csv", index=False)

# Step 8: Preview merged dataset
print("Merged dataset saved as 'cms_with_income_and_location.csv'")
print(merged_df[['ZIP', 'PROCEDURE', 'Avg_Mdcr_Pymt_Amt', 'Households Median Income (Dollars)', 'Affordability_Score']].head())

Merged dataset saved as 'cms_with_income_and_location.csv'
     ZIP                                        PROCEDURE  Avg_Mdcr_Pymt_Amt  \
0  36301  Level 2 Excision/ Biopsy/ Incision and Drainage         826.277954   
1  36301  Level 2 Excision/ Biopsy/ Incision and Drainage         826.277954   
2  36301  Level 2 Excision/ Biopsy/ Incision and Drainage         826.277954   
3  36301  Level 2 Excision/ Biopsy/ Incision and Drainage         826.277954   
4  36301  Level 2 Excision/ Biopsy/ Incision and Drainage         826.277954   

   Households Median Income (Dollars)  Affordability_Score  
0                             43054.0             0.019192  
1                             42045.0             0.019652  
2                             42140.0             0.019608  
3                             40702.0             0.020301  
4                             39618.0             0.020856  


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sentence_transformers import SentenceTransformer, util
from fuzzywuzzy import fuzz
import re

/opt/anaconda3/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [35]:
!pip install sentence-transformers
!python-Levenshtein

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: command not found: python-Levenshtein


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [69]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# Step 2: Load merged dataset
merged_df = pd.read_csv("cms_with_income_and_location.csv", low_memory=False)

# Step 3: Prepare data for model
# Drop rows with missing target or key features
model_df = merged_df.dropna(subset=['Avg_Mdcr_Pymt_Amt', 'ZIP', 'PROCEDURE', 'Households Median Income (Dollars)', 'population'])

# Drop leakage-prone features
model_df = model_df.drop(columns=['Avg_Mdcr_Alowd_Amt', 'Avg_Tot_Sbmtd_Chrgs'], errors='ignore')

# Clean column names to remove spaces and parentheses
model_df.columns = model_df.columns.str.replace('[\s\(\)]', '_', regex=True)

# Keep only numeric and boolean columns
model_df = model_df.select_dtypes(include=[np.number, 'bool']).copy()

# Step 4: Select features and target
target = 'Avg_Mdcr_Pymt_Amt'
features = [col for col in model_df.columns if col != target]
X = model_df[features]
y = model_df[target]

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train LightGBM model with optimized hyperparameters
model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    num_leaves=64,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)

# Step 7: Evaluate model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"LightGBM MAE on test set: ${mae:.2f}")




<>:21: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
/var/folders/qt/xn0rf1j15x3d_8zjxwk7w5_40000gn/T/ipykernel_19183/367870561.py:21: SyntaxWarning: invalid escape sequence '\s'
  model_df.columns = model_df.columns.str.replace('[\s\(\)]', '_', regex=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 3083782, number of used features: 7
[LightGBM] [Info] Start training from score 4451.498006
LightGBM MAE on test set: $65.61


In [91]:
# Step 8: Predict for a new example
# Define an example with mean values for context
example_input = X.mean().to_frame().T
example_prediction = model.predict(example_input)[0]
print(f"Predicted Medicare Payment for typical ZIP profile: ${example_prediction:.2f}")

# Step 9: Define custom prediction function
# Use mean values from training data to ensure same shape
mean_input = X.mean()

def predict_cost(zip_val, median_income, mean_income, pop, affordability):
    input_data = mean_input.copy()
    input_data['ZIP'] = zip_val
    input_data['Households_Median_Income_Dollars'] = median_income
    input_data['Households_Mean_Income_Dollars'] = mean_income
    input_data['population'] = pop
    input_data['Affordability_Score'] = affordability
    input_df = pd.DataFrame([input_data])[X.columns]  # Align with training features
    return model.predict(input_df)[0]

# Example prediction
cost = predict_cost(zip_val=10001, median_income=60000, mean_income=70000, pop=50000, affordability=0.08)
print(f"Predicted Medicare Payment for ZIP 10001: ${cost:.2f}")

# Step 10: Top 5 cheapest ZIPs for user profile
# Group by ZIP and calculate median features
grouped = merged_df.groupby("ZIP")[[
    "Households Median Income (Dollars)",
    "Households Mean Income (Dollars)",
    "population",
    "Affordability_Score"
]].median().dropna().reset_index()

def batch_predict(df):
    predictions = []
    for _, row in df.iterrows():
        pred = predict_cost(
            zip_val=row['ZIP'],
            median_income=row['Households Median Income (Dollars)'],
            mean_income=row['Households Mean Income (Dollars)'],
            pop=row['population'],
            affordability=row['Affordability_Score']
        )
        predictions.append(pred)
    df['Predicted_Cost'] = predictions
    return df

user_income = 65000
user_mean_income = 75000
user_pop = 40000
user_afford = 0.07

zip_group = merged_df.groupby("ZIP")[[
    "Households Median Income (Dollars)",
    "Households Mean Income (Dollars)",
    "population",
    "Affordability_Score"
]].median().dropna().reset_index()

zip_group['Predicted_Cost'] = zip_group.apply(
    lambda row: predict_cost(
        zip_val=row['ZIP'],
        median_income=user_income,
        mean_income=user_mean_income,
        pop=user_pop,
        affordability=user_afford
    ), axis=1
)

zip_predictions_sorted = zip_group.sort_values("Predicted_Cost")
print("Top 5 Cheapest ZIPs for User Profile:")
print(zip_predictions_sorted.head())

Predicted Medicare Payment for typical ZIP profile: $5100.32
Predicted Medicare Payment for ZIP 10001: $4588.89
Top 5 Cheapest ZIPs for User Profile:
        ZIP  Households Median Income (Dollars)  \
2838  94403                            103612.0   
2833  94115                             97090.0   
2834  94117                            122266.0   
2835  94133                             55098.0   
2836  94304                            104831.0   

      Households Mean Income (Dollars)  population  Affordability_Score  \
2838                          132788.0     43459.0             0.026351   
2833                          147143.0     32715.0             0.088874   
2834                          161068.0     37575.0             0.044786   
2835                          102837.0     24753.0             0.052863   
2836                          176181.0      4731.0             0.012593   

      Predicted_Cost  
2838     4028.083811  
2833     4028.083811  
2834     4028.083811  


In [95]:
!pip install geopandas


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 60.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.2 MB/s eta 0:00:00


In [99]:
# Step 11: Streamlit interactive visualization with filters and overlays
import streamlit as st
import pydeck as pdk

st.title("Medicare Cost Prediction Map")

# Sidebar filters
user_income = st.sidebar.slider("Median Income", 20000, 150000, 65000)
user_mean_income = st.sidebar.slider("Mean Income", 25000, 200000, 75000)
user_pop = st.sidebar.slider("Population", 1000, 1000000, 40000)
user_afford = st.sidebar.slider("Affordability Score", 0.01, 0.5, 0.07)

# Recalculate predictions based on inputs
zip_group['Predicted_Cost'] = zip_group.apply(
    lambda row: predict_cost(
        zip_val=row['ZIP'],
        median_income=user_income,
        mean_income=user_mean_income,
        pop=user_pop,
        affordability=user_afford
    ), axis=1
)

# Prepare merged map data
zip_latlng = merged_df[['ZIP', 'lat', 'lng']].dropna().drop_duplicates()
zip_latlng['ZIP'] = zip_latlng['ZIP'].astype(str).str.zfill(5)
zip_group['ZIP'] = zip_group['ZIP'].astype(str).str.zfill(5)
map_df = pd.merge(zip_group, zip_latlng, on='ZIP', how='left').dropna(subset=['lat', 'lng'])

st.subheader("Top 5 Cheapest ZIPs")
st.dataframe(map_df.sort_values("Predicted_Cost").head())

# Create interactive map
st.subheader("Predicted Medicare Costs Map")
st.pydeck_chart(pdk.Deck(
    map_style='mapbox://styles/mapbox/light-v9',
    initial_view_state=pdk.ViewState(
        latitude=37.5,
        longitude=-95.0,
        zoom=3.5,
        pitch=0,
    ),
    layers=[
        pdk.Layer(
            'ScatterplotLayer',
            data=map_df,
            get_position='[lng, lat]',
            get_color='[255 - (Predicted_Cost - 4000)/5, 100, 140]',
            get_radius=25000,
            pickable=True
        )
    ],
    tooltip={"text": "ZIP: {ZIP}\nCost: ${Predicted_Cost:.2f}"}
))


2025-05-13 00:11:25.030 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()